In [1]:
import os
import sys
sys.path.insert(0,'/home/t_goto/hf_env/lib/python3.10/site-packages')
import torch
import datasets
from datasets import load_from_disk
from transformers import (
    Trainer,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from trl import SFTTrainer
from peft import LoraConfig
from utils import InstructDataset, InstructCollator
from huggingface_hub import login
token = os.getenv('HF_TOKEN')
login(token)
load_data_flag = False # True if training data is reloaded

/home/t_goto/hf_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [2]:
os.environ.get('CUDA_VISIBLE_DEVICES')

'0'

In [3]:
compute_dtype = getattr(torch, "bfloat16")
quant_config = BitsAndBytesConfig(
    #llm_int8_threshold=200.0,
    #load_in_8bit=True,
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
    #bnb_8bit_use_double_quant=False, # need to avoid cast issue.
    #bnb_8bit_quant_type="nf8",
    #bnb_8bit_compute_dtype=compute_dtype,
    #llm_int8_skip_modules= ['decoder', 'lm_head', 'wo'],
)

In [4]:
model_name = "jetmoe/jetmoe-8b"
#model_name = "NousResearch/llama-2-7b-chat-hf"
#model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
#model_name = "microsoft/phi-1_5"
#model_name = "h2oai/h2o-danube2-1.8b-chat"
#model_name = "Aratako/Qwen1.5-MoE-2x7B"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    #device_map="auto",
    device_map = {"": torch.cuda.current_device()},
    quantization_config=quant_config
    # torch_dtype=torch.float16, # この時点でtorch.float16を指定すると、train時のlossが0.0になって学習がうまくいかない。原因がよくわかっていません。
)
model.config.use_cache = False # added in jetmoe
model.config.pretraining_tp = 1 # added in jetmoe

Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████| 4/4 [00:05<00:00,  1.43s/it]


In [5]:
med_data1=datasets.load_dataset("medalpaca/medical_meadow_mediqa")
train_dataset1 = InstructDataset((list(med_data1['train'])), tokenizer, ignore_index=tokenizer.pad_token_id)
len(train_dataset1)

100%|███████████████████████████████████████████████████████████████████████████████████████████| 2208/2208 [00:16<00:00, 135.08it/s]


2208

In [6]:
med_data2=datasets.load_dataset("medalpaca/medical_meadow_mmmlu")
train_dataset2 = InstructDataset((list(med_data2['train'])), tokenizer, ignore_index=tokenizer.pad_token_id)
len(train_dataset2)

100%|██████████████████████████████████████████████████████████████████████████████████████████| 3787/3787 [00:03<00:00, 1123.70it/s]


3787

In [7]:
med_data3=datasets.load_dataset("medalpaca/medical_meadow_wikidoc_patient_information")
train_dataset3 = InstructDataset((list(med_data3['train'])), tokenizer, ignore_index=tokenizer.pad_token_id)
len(train_dataset3)

100%|██████████████████████████████████████████████████████████████████████████████████████████| 5942/5942 [00:04<00:00, 1219.40it/s]


5942

In [8]:
med_data4=datasets.load_dataset("medalpaca/medical_meadow_pubmed_causal")
train_dataset4 = InstructDataset((list(med_data4['train'])), tokenizer, ignore_index=tokenizer.pad_token_id)
len(train_dataset4)

100%|██████████████████████████████████████████████████████████████████████████████████████████| 2446/2446 [00:01<00:00, 1225.92it/s]


2446

In [9]:
med_data5=datasets.load_dataset("medalpaca/medical_meadow_health_advice")
train_dataset5 = InstructDataset((list(med_data5['train'])), tokenizer, ignore_index=tokenizer.pad_token_id)
len(train_dataset5)

100%|██████████████████████████████████████████████████████████████████████████████████████████| 8676/8676 [00:06<00:00, 1293.53it/s]


8676

In [10]:
med_data6=datasets.load_dataset("medalpaca/medical_meadow_medical_flashcards")
train_dataset6 = InstructDataset((list(med_data6['train'])), tokenizer, ignore_index=tokenizer.pad_token_id)
len(train_dataset6)

100%|████████████████████████████████████████████████████████████████████████████████████████| 33955/33955 [00:28<00:00, 1212.37it/s]


33955

In [11]:
med_data7=datasets.load_dataset("medalpaca/medical_meadow_wikidoc")
train_dataset7 = InstructDataset((list(med_data7['train'])), tokenizer, ignore_index=tokenizer.pad_token_id)
len(train_dataset7)

100%|█████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:12<00:00, 810.58it/s]


10000

In [12]:
### skip this cell, dataset8 is not used to train actually.. skip this! ###
med_data8=datasets.load_dataset("medalpaca/medical_meadow_cord19")
train_dataset8 = InstructDataset((list(med_data8['train'])), tokenizer, ignore_index=tokenizer.pad_token_id)
len(train_dataset8)

 30%|█████████████████████████▋                                                             | 242665/821007 [11:01<26:15, 366.99it/s]


KeyboardInterrupt: 

In [13]:
train_dataset8

NameError: name 'train_dataset8' is not defined

In [14]:
med_data = load_from_disk("meadow_train")
train_dataset = InstructDataset(med_data, tokenizer, ignore_index=tokenizer.pad_token_id)

100%|███████████████████████████████████████████████████████████████████████████████████████████| 8142/8142 [00:15<00:00, 516.62it/s]


In [15]:
len(med_data)

8142

In [16]:
from torch.utils.data import ConcatDataset
med_datasets = ConcatDataset([train_dataset1,train_dataset2,train_dataset3,train_dataset4,train_dataset5,train_dataset6,train_dataset7,train_dataset])
len(med_datasets)

75156

In [18]:
from torch.utils.data import DataLoader

collator = InstructCollator(tokenizer, ignore_index=tokenizer.pad_token_id)
train_loader = DataLoader(med_datasets, collate_fn=collator, batch_size=4, shuffle=True)
#batch = next(iter(train_loader)) # for checking

In [19]:
load_data_flag

False

In [16]:
### This cell is not used for qlora training, skip this! ###
if load_data_flag:
    med_data=datasets.load_dataset("medalpaca/medical_meadow_medqa")
    med_data0=med_data['train'].train_test_split(test_size=0.2, shuffle=True)
    med_data0['train'].save_to_disk('meadow_train')
    med_data0['test'].save_to_disk('meadow_test')
    med_data = list(med_data0['train'])
    med_test = list(med_data0['test'])
else:
    train_data_name = 'meadow_train'
    if os.path.exists(train_data_name):
        med_data = load_from_disk(train_data_name)
    else:
        print('No training data: {train_data_name} exists.')

train_dataset = InstructDataset(med_data, tokenizer, ignore_index=tokenizer.pad_token_id)

100%|███████████████████████████████████████████████████████████████████████████████████████| 8142/8142 [00:15<00:00, 526.84it/s]


In [20]:
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=8,
    bias="none",
    task_type="CAUSAL_LM",
    #target_modules=['kv_proj', 'layer'] # need this only for jetmoe-8b
    target_modules=['kv_proj'] # need this only for jetmoe-8b
)

In [26]:
training_params = TrainingArguments(
    output_dir="./results_jetmoe-8b-4bit",
    #output_dir="./results_llama2-7b-more_max",
    #output_dir="./results_tiny_llama-1.1b",
    #output_dir="./results_phi-1_5",
    #output_dir="./results_jetmoe_more_max",
    #num_train_epochs=0.2, # epoch 3758 too long
    num_train_epochs=0.05, # epoch 940, 
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    #report_to="tensorboard"
)

In [22]:
len(med_datasets)

75156

In [27]:
collator = InstructCollator(tokenizer, ignore_index=tokenizer.pad_token_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
trainer = SFTTrainer(
    model=model,
    train_dataset=med_datasets,
    data_collator=collator,
    peft_config=peft_params,
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=True,
)

/home/t_goto/hf_env/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:246: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


In [28]:
import time
start = time.perf_counter()
trainer.train()
end = time.perf_counter()
print(f'{end-start} [sec]')
trainer.model.save_pretrained("llama2_FT_train_adapter1")

Step,Training Loss
25,4.279900
50,0.497000
75,0.707900
100,0.266200
125,0.597400
150,0.229600
175,0.621900
200,0.178100
225,0.584500
250,0.232100


/home/t_goto/hf_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/t_goto/hf_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/t_goto/hf_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/t_goto/hf_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` 

1063.4493739623576 [sec]


/home/t_goto/hf_env/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [21]:
torch.cuda.current_device()

0

In [19]:
torch.cuda.empty_cache()

In [10]:
import pandas as pd

In [13]:
df=pd.read_csv('gpu_jetmoe.csv',sep=',')

In [17]:
df[0:3].mean()

TypeError: Could not convert ['2024/07/01 03:43:31.2672024/07/01 03:43:33.2692024/07/01 03:43:35.271'
 ' NVIDIA A100 80GB PCIe NVIDIA A100 80GB PCIe NVIDIA A100 80GB PCIe'
 ' 0 % 0 % 0 %' ' 663 MiB 663 MiB 663 MiB'] to numeric

In [39]:
df[0:3]

,timestamp,name,utilization.gpu [%],memory.used [MiB]
0,2024/07/01 03:13:19.557,NVIDIA A100 80GB PCIe,0 %,663 MiB
1,2024/07/01 03:13:21.559,NVIDIA A100 80GB PCIe,0 %,663 MiB
2,2024/07/01 03:13:23.560,NVIDIA A100 80GB PCIe,0 %,663 MiB


In [57]:
df['name']

KeyError: 'name'